In [1]:
# Imports
import os
from glob import glob
import pandas as pd
import numpy as np
import scipy.optimize as opt
import matplotlib.pyplot as plt
import matplotlib as mpl
import scipy.stats.qmc as qmc

import warnings
warnings.simplefilter("ignore", category=RuntimeWarning) # for RuntimeWarning only

In [2]:
def read_INP_data(filename, conc_suffix):
    
    data = pd.read_csv(filename,index_col='temp_bin')
    # If you want to mask any of the data to remove particular runs, you need to add this here.
    # It is easy to filter so that you only get columns that e.g. contain "CAO" or have
    # heights > 2000m if they're in the column name.
    if conc_suffix == '_NT':
        err_suffix = '_err'
    else:
        err_suffix = conc_suffix+'_err'
    raw_concs = data[data.columns[pd.Series(data.columns).str.endswith(conc_suffix)]]
    raw_concs.columns = raw_concs.columns.str.replace(conc_suffix,'')
    logged_concs = np.log(raw_concs)


    raw_errors = data[data.columns[pd.Series(data.columns).str.endswith(err_suffix)]]
    raw_errors.columns = raw_errors.columns.str.replace(err_suffix,'')
    temps = logged_concs.index.values
    return temps, raw_concs, logged_concs, raw_errors

In [3]:
def four_param_fit(T, nu, T_max, a, b):
    # Apply the fit to an array of temperatures T
    return nu + a*(T_max-T)**b

In [4]:
def strip_nans(*args):
    # Strip nans from multiple inputs where NaN values have the same indices in each input.
    nan_mask = np.logical_not(np.isnan(args[0]))
    nanless = []
    for a in args:
        nanless.append(a[nan_mask])
    return nanless

In [5]:
def reduced_chi_squared_test(p, temps, log_concs, weighted_error):
    # Perform reduced chi-squared test for fit parameters p
    modelled = four_param_fit(temps, *p) #*p unlocks the list p so this is like writing temps, p[0], p[1]...
    residual = log_concs - modelled
    chi_sq = np.sum((residual/weighted_error)**2)
    red_chi_sq = chi_sq/(len(log_concs)-len(p))
    return red_chi_sq

In [6]:
# Create 4-dimensional latin hypercube object. Seed chosen for replication of results.
lhc = qmc.LatinHypercube(4, seed=280299)
# Take 100 LH samples
samples = lhc.random(n=100)

In [7]:
def bounded_and_sampled_scipy_4_param(temps, logged_concs, raw_concs, raw_errors, samples, run_id,
                                      nu_range=(-6,-2), plot=False, silent=True):
    """
    Ugly but functional function to find potential best-fits.
    
    Parameters
    ----------
    temps: ndarray
        Temperature bins
    logged_concs: pandas DataFrame
        concentrations with a natural logarithm applied
    raw_concs: pandas DataFrame
        concentrations
    raw_errors: pandas DataFrame
        errors
    samples: qmc Object
        samples chosen in hypercube space
    run_id: string
        identifier of the run
    nu_range: 2-tuple
        (lowest possible nu value, highest possible nu value)
    plot: boolean
        not yet implemented
    silent: boolean
        say when convergence fails (likely to happen often among the 100 samples)

    Returns
    -------
    pandas
        all derived parameters for the run that converged
    """

    # Create dataframe to hold parameters for the run                                      
    run_params = pd.DataFrame(columns=['nu','T_max','a','b','X2','cov'])
    # Strip nans where appropriate as curve_fit doesn't handle them
    nanless_concs, nanless_raw, nanless_errs, nanless_temps = strip_nans(
                                                        logged_concs[run_id].values,
                                                        raw_concs[run_id].values,
                                                        raw_errors[run_id].values,
                                                        temps
    )
    # Weight errors in logarithmic space
    nanless_weights = nanless_errs/nanless_raw

    # Set bounds of the sample space
    lower_bounds = [nu_range[0], max(nanless_temps)-2, 0.3, 0.5]
    upper_bounds = [nu_range[1], max(nanless_temps)+2, 1.05, 1.0]

    # Apply the LH samples to the sample space
    scaled = qmc.scale(samples, lower_bounds, upper_bounds)

    # Loop through all of the samples and attempt to find a best-fit
    for ps in scaled:
        try:
            p, c = opt.curve_fit(four_param_fit, nanless_temps, nanless_concs, p0=ps,
                sigma=nanless_weights, bounds=(lower_bounds,upper_bounds), maxfev=20000)
            # Calculate goodness of fit parameter
            X2 = reduced_chi_squared_test(p, nanless_temps, nanless_concs, nanless_weights)
            # Store in the in the dataframe
            run_params.loc[len(run_params)] = [p[0], p[1], p[2], p[3], X2, c]
        except:
            # Do not fail if convergence can't be found.
            if not silent:
                print('Failed to converge with parameters', ps)
    return run_params

In [8]:
def get_parameter_df(temps, logged_concs, raw_concs, raw_errors, samples, nu_range=(-6,-2)):
    """
    Get a dataframe with the parameters that have the lowest reduced chi-squared for each run.

    Parameters
    ----------
    temps: ndarray
        Temperature bins
    logged_concs: pandas DataFrame
        concentrations with a natural logarithm applied
    raw_concs: pandas DataFrame
        concentrations
    raw_errors: pandas DataFrame
        errors
    samples: qmc Object
        samples chosen in hypercube space
    nu_range: 2-tuple
        (lowest possible nu value, highest possible nu value)

    Returns
    -------
    pandas Dataframe
        best parameters for each run
    """
    bounded_params = {}
    for run in logged_concs:
        # Loop through every run to get all parameters and add to dictionary
        bounded_params[run] = bounded_and_sampled_scipy_4_param(temps, logged_concs, raw_concs,
                                                                raw_errors, samples, run, nu_range)

    # create an empty dataframe to store the lowest X2 values
    lowest_X2_bounded_df = pd.DataFrame(columns=['nu','T_max','a','b','X2','cov'])

    # loop through the dictionary and find the lowest X2 value for each run
    for run_id, df in bounded_params.items():
        df = df.dropna()
        if df.empty:
            # create a new row with NaN values and concatenate it to lowest_X2_df
            lowest_X2_bounded_row = pd.DataFrame([[np.nan]*len(lowest_X2_bounded_df.columns)], 
                                        columns=lowest_X2_bounded_df.columns, 
                                        index=[run_id])
            lowest_X2_bounded_df = pd.concat([lowest_X2_bounded_df, lowest_X2_bounded_row])
        else:
            lowest_X2_bounded_row = df.nsmallest(1, 'X2')
            lowest_X2_bounded_row.index = [run_id]
            lowest_X2_bounded_df = pd.concat([lowest_X2_bounded_df, lowest_X2_bounded_row])

    return lowest_X2_bounded_df

In [9]:
base = '/home/users/erinraif/mphase_data/inp_data/'
suffix = '_NT'
INP_concs = 'mphase_ER_CAOs.csv'

In [11]:
nu_range = (-6,-2)

In [15]:
params_df = {}
temps, raw_concs, logged_concs, raw_errors = read_INP_data(base+INP_concs, suffix)
params = get_parameter_df(temps, logged_concs, raw_concs, raw_errors, samples, nu_range)
params_df[suffix[1:]] = params

In [16]:
params_df['n_INP'] = params_df.pop('NT')


In [17]:
dfs_with_suffixes = []

# Loop through the dictionary and add suffixes to the columns
for key, df in params_df.items():
    # Add suffixes to the columns
    df = df.add_suffix(f'_{key}')
    # Append the modified DataFrame to the list
    dfs_with_suffixes.append(df)

# Concatenate the modified DataFrames horizontally
export_params = pd.concat(dfs_with_suffixes, axis=1)

In [18]:
export_params.index.name = 'unique_ID'
export_params = export_params.loc[:, ~export_params.columns.str.contains('X2|cov')]


In [20]:
export_params.to_csv('/home/users/erinraif/mphase_data/metadata/parametrisations.csv')

In [21]:
def general_parametrisation(temps, logged_concs, raw_concs, raw_errors, samples):
    """
    Get a dataframe with the parameters that have the lowest reduced chi-squared for each run.

    Parameters
    ----------
    temps: ndarray
        Temperature bins
    logged_concs: pandas DataFrame
        concentrations with a natural logarithm applied
    raw_concs: pandas DataFrame
        concentrations
    raw_errors: pandas DataFrame
        errors
    samples: qmc Object
        samples chosen in hypercube space
    nu_range: 2-tuple
        (lowest possible nu value, highest possible nu value)

    Returns
    -------
    pandas Dataframe
        best parameters for each run
    """
    logged_conc_values = logged_concs.values.flatten()
    raw_errors_values = raw_errors.values.flatten()
    raw_concs_values = raw_concs.values.flatten()
    temp_bin_values = logged_concs.index.repeat(len(logged_concs.columns)).values

    # Remove NaN values and create arrays
    logged_conc_arr = logged_conc_values[~np.isnan(logged_conc_values)]
    raw_errors_arr = raw_errors_values[~np.isnan(raw_errors_values)]
    raw_concs_arr = raw_concs_values[~np.isnan(raw_concs_values)]
    weight_arr = raw_errors_arr/raw_concs_arr
    temp_bin_arr = temp_bin_values[~np.isnan(logged_conc_values)]

    lower_bounds = [-6, -9, 0.3, 0.5]
    upper_bounds = [-2, -5, 1.05, 1.0]
    run_params = pd.DataFrame(columns=['nu','T_max','a','b','X2','cov'])
    scaled = qmc.scale(samples, lower_bounds, upper_bounds)
    for ps in scaled:
        try:
            p, c = opt.curve_fit(four_param_fit, temp_bin_arr, logged_conc_arr, p0=ps,
                                    sigma=weight_arr, bounds=(lower_bounds,upper_bounds), maxfev=20000)
            X2 = reduced_chi_squared_test(p, temp_bin_arr, logged_conc_arr, weight_arr)
            run_params.loc[len(run_params)] = [p[0], p[1], p[2], p[3], X2, c]
        except:
            pass
            #print('Failed to converge with parameters', ps)

    # Find row with lowest X2
    min_row_index = run_params['X2'].idxmin()

    # Now you can access the entire row using iloc
    min_row = run_params.iloc[min_row_index]
    return min_row


In [22]:
temps, raw_concs, logged_concs, raw_errors = read_INP_data(base+INP_concs, '_NT')
gen_param = general_parametrisation(temps, logged_concs, raw_concs, raw_errors, samples)
gen_param

nu                                               -5.802257
T_max                                            -8.499517
a                                                 1.049948
b                                                 0.646522
X2                                                6.031805
cov      [[1.4508662509219334, -0.23030623389757995, -0...
Name: 0, dtype: object

In [32]:
def general_parametrisation_minus6(temps, logged_concs, raw_concs, raw_errors, samples):
    """
    Get a dataframe with the parameters that have the lowest reduced chi-squared for each run.

    Parameters
    ----------
    temps: ndarray
        Temperature bins
    logged_concs: pandas DataFrame
        concentrations with a natural logarithm applied
    raw_concs: pandas DataFrame
        concentrations
    raw_errors: pandas DataFrame
        errors
    samples: qmc Object
        samples chosen in hypercube space
    nu_range: 2-tuple
        (lowest possible nu value, highest possible nu value)

    Returns
    -------
    pandas Dataframe
        best parameters for each run
    """
    logged_conc_values = logged_concs.values.flatten()
    raw_errors_values = raw_errors.values.flatten()
    raw_concs_values = raw_concs.values.flatten()
    temp_bin_values = logged_concs.index.repeat(len(logged_concs.columns)).values

    # Remove NaN values and create arrays
    logged_conc_arr = logged_conc_values[~np.isnan(logged_conc_values)]
    raw_errors_arr = raw_errors_values[~np.isnan(raw_errors_values)]
    raw_concs_arr = raw_concs_values[~np.isnan(raw_concs_values)]
    weight_arr = raw_errors_arr/raw_concs_arr
    temp_bin_arr = temp_bin_values[~np.isnan(logged_conc_values)]

    lower_bounds = [-7, -13, 0.3, 0.3]
    upper_bounds = [-3, -11, 1.4, 1.0]
    run_params = pd.DataFrame(columns=['nu','T_max','a','b','X2','cov'])
    scaled = qmc.scale(samples, lower_bounds, upper_bounds)
    for ps in scaled:
        try:
            p, c = opt.curve_fit(four_param_fit, temp_bin_arr, logged_conc_arr, p0=ps,
                                    #sigma=weight_arr, bounds=(lower_bounds,upper_bounds), maxfev=20000)
                                     bounds=(lower_bounds,upper_bounds), maxfev=20000)
            X2 = reduced_chi_squared_test(p, temp_bin_arr, logged_conc_arr, weight_arr)
            run_params.loc[len(run_params)] = [p[0], p[1], p[2], p[3], X2, c]
        except:
            pass
            #print('Failed to converge with parameters', ps)
    print(run_params)
    # Find row with lowest X2
    min_row_index = run_params['X2'].idxmin()

    # Now you can access the entire row using iloc
    min_row = run_params.iloc[min_row_index]
    return min_row


In [37]:
temps, raw_concs, logged_concs, raw_errors = read_INP_data(base+INP_concs, '_NT')


gen_param = general_parametrisation_minus6(temps[(temps < -11) & (temps > -28)],
                                           logged_concs[(temps < -11) & (temps > -28)],
                                           raw_concs[(temps < -11) & (temps > -28)], 
                                           raw_errors[(temps < -11) & (temps > -28)], 
                                           lhc.random(1000))
gen_param

           nu  T_max         a         b        X2  \
0   -4.990384  -11.0  0.828560  0.709489  9.470812   
1   -4.990389  -11.0  0.828564  0.709488  9.470809   
2   -4.990381  -11.0  0.828558  0.709490  9.470814   
3   -4.990412  -11.0  0.828580  0.709482  9.470796   
4   -4.990393  -11.0  0.828566  0.709487  9.470807   
..        ...    ...       ...       ...       ...   
226 -4.990385  -11.0  0.828561  0.709489  9.470811   
227 -4.990366  -11.0  0.828548  0.709494  9.470824   
228 -4.990386  -11.0  0.828561  0.709489  9.470811   
229 -4.990391  -11.0  0.828565  0.709488  9.470807   
230 -4.990396  -11.0  0.828568  0.709486  9.470805   

                                                   cov  
0    [[2.0798874072228366, -2.4040759803944574, -0....  
1    [[2.0798830010179774, -2.4040582262595853, -0....  
2    [[2.0798901972816677, -2.4040860251018046, -0....  
3    [[2.0798634516745436, -2.403982245419489, -0.6...  
4    [[2.07988035300145, -2.404047425086673, -0.632...  
..       

nu                                               -4.990412
T_max                                                -11.0
a                                                  0.82858
b                                                 0.709482
X2                                                9.470796
cov      [[2.0798634516745436, -2.403982245419489, -0.6...
Name: 3, dtype: object

In [52]:
logged_concs[temps < -6]

,c271r1t,c271r2t,c271r3t,c272r6t,c273r1t,c273r2t,c274r1t,c274r3t,c275r1t,c276r2t,...,c278r4t,c279r1t,c279r2t,c280r1t,c280r2t,c280r3t,c280r4t,c282r1t,c282r2t,c282r3t
temp_bin,,,,,,,,,,,,,,,,,,,,,
-7,-4.374509,-3.625851,NaN,NaN,-4.219966,NaN,-2.660378,-4.226129,-4.058575,-3.141419,...,-4.291410,-4.464432,-3.909157,-4.082317,-3.065772,-3.792827,NaN,-3.746145,-3.367754,-4.289627
-8,-4.374509,-2.773635,NaN,-3.528545,-4.219966,NaN,-2.660378,-4.226129,-2.778406,-2.269370,...,-3.160343,-4.464432,-3.201824,-3.381072,-2.534326,-3.792827,-3.522101,-3.746145,-2.653146,-2.883828
-9,-2.251189,-2.294661,-2.629509,NaN,-3.134426,NaN,-1.791597,-3.115174,-2.217172,-1.458899,...,-2.442133,NaN,-2.360537,-2.985480,-2.041293,-2.141269,NaN,-3.078006,-2.239815,-2.313515
-10,-1.919870,-1.831828,-2.461277,NaN,NaN,NaN,-1.130885,-2.605526,-2.063378,-1.030814,...,-1.688445,-3.093874,-1.664554,-2.111545,-1.808779,-1.653586,NaN,-2.150961,-1.949926,-1.948619
-11,-1.592346,-1.482243,-2.341486,NaN,-2.684639,NaN,-0.913499,-2.461927,-1.572078,-0.707824,...,-1.525718,-2.141595,-1.317919,-1.638857,-1.444508,-1.223534,-2.439651,-1.131588,-1.434776,-1.777586
-12,-1.391737,-1.224018,-1.749884,-3.229703,-2.199993,NaN,-0.599286,-2.324901,-1.208326,-0.432828,...,-1.455394,-1.547038,-0.939612,-1.141412,-1.153308,-1.147179,-2.152172,-0.359800,-0.999533,-1.540964
-13,-1.116404,-0.876459,-1.706907,-2.842569,-1.704040,-4.449084,-0.350649,-1.745675,-1.071683,-0.195410,...,-1.115944,-1.446346,-0.352640,-0.566297,-0.692740,-0.782072,-1.547888,0.279628,-0.786236,-1.297345
-14,-0.695577,-0.812345,-1.424650,-2.841209,NaN,-3.884998,-0.088316,-1.617030,-0.948806,0.092887,...,-0.608423,-1.182363,-0.054039,-0.245626,-0.357204,-0.490804,-1.075218,0.497383,-0.500269,-1.040551
-15,-0.585214,-0.707671,-1.123798,-2.242103,-1.523561,-3.534395,0.066860,-1.122683,-0.796478,0.415782,...,-0.361512,-0.717732,0.238673,-0.032863,0.056949,-0.130774,-0.985820,0.758587,-0.199877,-0.589001
